# 选取每个industry top3的股票来建立portfolio

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
# Top 3 stocks from each GICS sector
tickers = [
    "AAPL", "MSFT", "NVDA",  # Technology
    "JNJ", "PFE", "UNH",     # Healthcare
    "JPM", "BAC", "WFC",     # Financials
    "XOM", "CVX", "COP",     # Energy
    "PG", "KO", "PEP",       # Consumer Staples
    "HD", "LOW", "TGT",      # Consumer Discretionary
    "NEE", "DUK", "SO",      # Utilities
    "GOOGL", "META", "DIS",  # Communication Services
    "UNP", "HON", "RTX",     # Industrials
    "SHW", "LIN", "FCX",     # Materials
    "AMT", "PLD", "EQIX"     # Real Estate
]

# Download full multi-indexed data
data = yf.download(tickers, start="2020-01-01", end="2024-12-31", group_by='ticker', auto_adjust=True)

# Extract "Adj Close" prices correctly for all tickers
adj_close = pd.concat([data[ticker]['Close'] for ticker in tickers], axis=1)
adj_close.columns = tickers

# Calculate daily returns
returns = adj_close.pct_change().dropna()

# Save or display
returns.to_csv("sector_portfolio_returns.csv")
print("✅ Daily returns calculated and saved.")


[*********************100%***********************]  33 of 33 completed


✅ Daily returns calculated and saved.


# 现在首先计算等权重投资组合的年年回报率和年波动率（hold不变，作为benchmark），然后使用传统的mean variance法，每天和每周、每月调整投资组合，并计算该投资组合的年回报率、年波动率。用作benchmark

In [10]:
n_stocks = returns.shape[1]
equal_weights = np.ones(n_stocks) / n_stocks
equal_portfolio_returns = returns.dot(equal_weights)

annual_return = (1 + equal_portfolio_returns.mean()) ** 252 - 1
annual_volatility = equal_portfolio_returns.std() * np.sqrt(252)
risk_free_rate = 0.02
sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

print(f"Equal-Weighted Portfolio")
print(f"Annual Return: {annual_return:.4f}")
print(f"Annual Volatility: {annual_volatility:.4f}")
print(f"Sharpe Ratio (Rf=2%): {sharpe_ratio:.4f}")



Equal-Weighted Portfolio
Annual Return: 0.1841
Annual Volatility: 0.2134
Sharpe Ratio (Rf=2%): 0.7690


In [11]:
import pandas as pd
import numpy as np
import cvxpy as cp

# === Load return data ===
returns = pd.read_csv("sector_portfolio_returns.csv", index_col=0, parse_dates=True)
returns = returns.dropna(axis=1)

# === Define Parameters and Data class ===
class Parameters:
    def __init__(self, n_assets):
        self.w_min = np.zeros(n_assets)
        self.w_max = np.ones(n_assets)
        self.c_min = -0.1
        self.c_max = 0.1
        self.L_tar = 1.0
        self.T_tar = 0.2
        self.z_min = -0.2 * np.ones(n_assets)
        self.z_max = 0.2 * np.ones(n_assets)
        self.gamma_hold = 0.01
        self.gamma_trade = 0.01
        self.risk_target = 0.04

class Data:
    def __init__(self, mean, cov, w_prev, kappa_short, kappa_borrow, kappa_spread, kappa_impact):
        self.mean = mean
        self.cov = cov
        self.w_prev = w_prev
        self.kappa_short = kappa_short
        self.kappa_borrow = kappa_borrow
        self.kappa_spread = kappa_spread
        self.kappa_impact = kappa_impact

# === DCP Failure Counter ===
dcp_fail_count = 0

# === Markowitz Optimizer ===
def markowitz(data: Data, param: Parameters, verbose=False):
    global dcp_fail_count
    n_assets = param.w_min.shape[0]
    w, c = cp.Variable(n_assets), cp.Variable()
    z = w - data.w_prev
    T = cp.norm1(z) / 2
    L = cp.norm1(w)

    risk = cp.quad_form(w, data.cov)
    ret = w.T @ data.mean

    holding_cost = data.kappa_short @ cp.pos(-w) + data.kappa_borrow * cp.pos(-c)
    trading_cost = data.kappa_spread @ cp.abs(z) + data.kappa_impact @ cp.power(cp.abs(z), 3 / 2)

    objective = ret - param.gamma_hold * holding_cost - param.gamma_trade * trading_cost

    constraints = [
        cp.sum(w) + c == 1,
        param.w_min <= w,
        w <= param.w_max,
        L <= param.L_tar,
        param.c_min <= c,
        c <= param.c_max,
        param.z_min <= z,
        z <= param.z_max,
        T <= param.T_tar,
        risk <= param.risk_target
    ]

    problem = cp.Problem(cp.Maximize(objective), constraints)

    if not problem.is_dcp():
        dcp_fail_count += 1
        if verbose:
            print("DCP check failed.")
        return None

    try:
        problem.solve()
        return w.value
    except Exception as e:
        if verbose:
            print(f"Solver error: {e}")
        return None

# === Rebalancing Simulation ===
def simulate_markowitz_rebalancing(returns, rebalance_freq='M', lookback=60):
    returns = returns.dropna(axis=1)
    dates = returns.index
    n_assets = returns.shape[1]

    w_prev = np.ones(n_assets) / n_assets
    portfolio_returns = []

    kappa_short = 0.001 * np.ones(n_assets)
    kappa_borrow = 0.001
    kappa_spread = 0.005 * np.ones(n_assets)
    kappa_impact = 0.01 * np.ones(n_assets)
    param = Parameters(n_assets)

    for i in range(lookback, len(returns)):
        date = dates[i]
        rebalance = (
            rebalance_freq == 'D' or
            (rebalance_freq == 'W' and date.weekday() == 0) or
            (rebalance_freq == 'M' and date.day == 1)
        )

        if rebalance:
            window = returns.iloc[i - lookback:i]
            mu = window.mean().values * 252
            cov = window.cov().values * 252
            data = Data(mu, cov, w_prev, kappa_short, kappa_borrow, kappa_spread, kappa_impact)
            w_opt = markowitz(data, param)
            if w_opt is not None:
                w_prev = w_opt

        daily_return = returns.iloc[i].values @ w_prev
        portfolio_returns.append(daily_return)

    return pd.Series(portfolio_returns, index=returns.index[lookback:])

# === Run simulations ===
mvo_daily = simulate_markowitz_rebalancing(returns, 'D')
mvo_weekly = simulate_markowitz_rebalancing(returns, 'W')
mvo_monthly = simulate_markowitz_rebalancing(returns, 'M')

# === Calculate annual metrics and Sharpe ratio ===
def metrics_with_sharpe(r):
    ann_return = (1 + r.mean()) ** 252 - 1
    ann_vol = r.std() * np.sqrt(252)
    sharpe = ann_return / ann_vol if ann_vol != 0 else np.nan
    return ann_return, ann_vol, sharpe

metrics = pd.DataFrame({
    "Strategy": ["MVO Daily", "MVO Weekly", "MVO Monthly"],
    "Annual Return": [metrics_with_sharpe(mvo_daily)[0],
                      metrics_with_sharpe(mvo_weekly)[0],
                      metrics_with_sharpe(mvo_monthly)[0]],
    "Annual Volatility": [metrics_with_sharpe(mvo_daily)[1],
                          metrics_with_sharpe(mvo_weekly)[1],
                          metrics_with_sharpe(mvo_monthly)[1]],
    "Sharpe Ratio": [metrics_with_sharpe(mvo_daily)[2],
                     metrics_with_sharpe(mvo_weekly)[2],
                     metrics_with_sharpe(mvo_monthly)[2]]
})

print(metrics)
print(f"Total DCP check failed: {dcp_fail_count}")


/usr/local/lib/python3.11/dist-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


      Strategy  Annual Return  Annual Volatility  Sharpe Ratio
0    MVO Daily       0.236335           0.218295      1.082641
1   MVO Weekly       0.185007           0.218523      0.846625
2  MVO Monthly       0.402794           0.225048      1.789812
Total DCP check failed: 0


# 用新的风险控制代替variance，这里我换了VIX 用industry sentimental index试试，其他的还有CVAR 和 HMM

In [12]:
import pandas as pd
import numpy as np
import yfinance as yf

# === Step 1: ETF 对应行业 ===
sector_etfs = {
    "Technology": "XLK",
    "Healthcare": "XLV",
    "Financials": "XLF",
    "Energy": "XLE",
    "Consumer Staples": "XLP",
    "Consumer Discretionary": "XLY",
    "Utilities": "XLU",
    "Communication Services": "XLC",
    "Industrials": "XLI",
    "Materials": "XLB",
    "Real Estate": "XLRE"
}

# === Step 2: 股票 -> 行业 映射 ===
stock_sector_map = {
    "AAPL": "Technology", "MSFT": "Technology", "NVDA": "Technology",
    "JNJ": "Healthcare", "PFE": "Healthcare", "UNH": "Healthcare",
    "JPM": "Financials", "BAC": "Financials", "WFC": "Financials",
    "XOM": "Energy", "CVX": "Energy", "COP": "Energy",
    "PG": "Consumer Staples", "KO": "Consumer Staples", "PEP": "Consumer Staples",
    "HD": "Consumer Discretionary", "LOW": "Consumer Discretionary", "TGT": "Consumer Discretionary",
    "NEE": "Utilities", "DUK": "Utilities", "SO": "Utilities",
    "GOOGL": "Communication Services", "META": "Communication Services", "DIS": "Communication Services",
    "UNP": "Industrials", "HON": "Industrials", "RTX": "Industrials",
    "SHW": "Materials", "LIN": "Materials", "FCX": "Materials",
    "AMT": "Real Estate", "PLD": "Real Estate", "EQIX": "Real Estate"
}

# === Step 3: 下载 ETF 数据 ===
etf_tickers = list(sector_etfs.values())
etf_data = yf.download(etf_tickers, start="2020-01-01", end="2024-12-31", auto_adjust=True)
etf_prices = etf_data['Close']
etf_returns = etf_prices.pct_change().dropna()

# === Step 4: 滚动年化波动率 (21日窗口)
vol_window = 21
rolling_vol = etf_returns.rolling(window=vol_window).std() * np.sqrt(252)

# === Step 5: 计算 rolling z-score （每日）
zscore_daily = (rolling_vol - rolling_vol.mean()) / rolling_vol.std()
zscore_daily = zscore_daily.dropna()

# === Step 6: 提取每周（周一）、每月（1号）z-score
zscore_weekly = zscore_daily[zscore_daily.index.weekday == 0]
zscore_monthly = zscore_daily[zscore_daily.index.day == 1]

# === Step 7: 创建函数：将行业z-score映射到股票 ===
def map_zscore_to_stocks(zscore_df):
    result = {}
    for date, row in zscore_df.iterrows():
        mapped = {
            stock: row[sector_etfs[stock_sector_map[stock]]]
            for stock in stock_sector_map
        }
        result[date] = mapped
    return pd.DataFrame(result).T.sort_index()

# === Step 8: 映射成股票层面的情绪时间序列 ===
sentiment_daily = map_zscore_to_stocks(zscore_daily)
sentiment_weekly = map_zscore_to_stocks(zscore_weekly)
sentiment_monthly = map_zscore_to_stocks(zscore_monthly)

# === Step 9: 每一行进行 Min-Max 归一化（0~1）
def min_max_scale(df):
    return (df.T - df.min(axis=1)) / (df.max(axis=1) - df.min(axis=1) + 1e-8)  # 防除零
sentiment_daily = min_max_scale(sentiment_daily).T
sentiment_weekly = min_max_scale(sentiment_weekly).T
sentiment_monthly = min_max_scale(sentiment_monthly).T

# === Step 10: 保存 CSV（可选）
sentiment_daily.to_csv("sector_sentiment_daily_scaled.csv")
sentiment_weekly.to_csv("sector_sentiment_weekly_scaled.csv")
sentiment_monthly.to_csv("sector_sentiment_monthly_scaled.csv")

# === Step 11: 预览结果 ===
print("股票行业情绪 Min-Max 归一化 z-score 示例（每日）:")
print(sentiment_daily.head())


[*********************100%***********************]  11 of 11 completed


股票行业情绪 Min-Max 归一化 z-score 示例（每日）:
                AAPL      MSFT      NVDA  JNJ  PFE  UNH       JPM       BAC  \
2020-02-03  0.753416  0.753416  0.753416  1.0  1.0  1.0  0.769407  0.769407   
2020-02-04  0.720105  0.720105  0.720105  1.0  1.0  1.0  0.626125  0.626125   
2020-02-05  0.612876  0.612876  0.612876  1.0  1.0  1.0  0.646808  0.646808   
2020-02-06  0.617027  0.617027  0.617027  1.0  1.0  1.0  0.635209  0.635209   
2020-02-07  0.630908  0.630908  0.630908  1.0  1.0  1.0  0.614805  0.614805   

                 WFC       XOM  ...       DIS       UNP       HON       RTX  \
2020-02-03  0.769407  0.377857  ...  0.503079  0.798000  0.798000  0.798000   
2020-02-04  0.626125  0.253226  ...  0.377655  0.810091  0.810091  0.810091   
2020-02-05  0.646808  0.522986  ...  0.249631  0.779470  0.779470  0.779470   
2020-02-06  0.635209  0.527689  ...  0.309166  0.781298  0.781298  0.781298   
2020-02-07  0.614805  0.491884  ...  0.280278  0.783123  0.783123  0.783123   

               

In [13]:
# === Step 2: CVaR 函数 ===
def compute_cvar_series(returns_df, weights, window=21, alpha=0.05):
    """
    Compute rolling portfolio CVaR using a fixed lookback window.
    - returns_df: pd.DataFrame (daily asset returns)
    - weights: np.array (portfolio weights)
    - window: lookback window (default 21 days)
    - alpha: tail quantile (default 5%)
    """
    port_returns = returns_df @ weights
    cvar_series = port_returns.rolling(window=window).apply(
        lambda x: -np.mean(x[x < np.quantile(x, alpha)]) if len(x[x < np.quantile(x, alpha)]) > 0 else np.nan,
        raw=False
    )
    return cvar_series.dropna()

# === Step 3: 设置等权重 ===
n_assets = returns.shape[1]
equal_weights = np.ones(n_assets) / n_assets

# === Step 4: 计算每日 CVaR（21日窗口）===
cvar_daily = compute_cvar_series(returns, equal_weights, window=21, alpha=0.05)

# === Step 5: 筛选每周/每月 ===
cvar_weekly = cvar_daily[cvar_daily.index.weekday == 0]     # 每周一
cvar_monthly = cvar_daily[cvar_daily.index.day == 1]        # 每月第一天

# === Step 6: 合并成 DataFrame 并保存（可视化用）===
cvar_df = pd.DataFrame({
    "CVaR_Daily": cvar_daily,
    "CVaR_Weekly": cvar_weekly,
    "CVaR_Monthly": cvar_monthly
})
# 保存到 CSV 文件
cvar_df.to_csv("cvar_frequencies.csv")

# 可选：预览
print("CVaR 文件已保存: cvar_frequencies.csv")
print(cvar_df.head())

CVaR 文件已保存: cvar_frequencies.csv
            CVaR_Daily  CVaR_Weekly  CVaR_Monthly
Date                                             
2020-02-03    0.020745     0.020745           NaN
2020-02-04    0.020745          NaN           NaN
2020-02-05    0.020745          NaN           NaN
2020-02-06    0.020745          NaN           NaN
2020-02-07    0.020745          NaN           NaN


In [15]:
pip install hmmlearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 4.3 MB/s eta 0:00:00


In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt

# === Step 1: 下载市场代表性资产（S&P500） ===
sp500 = yf.download("^GSPC", start="2020-01-01", end="2024-12-31", auto_adjust=True)['Close']
sp500_returns = sp500.pct_change().dropna().values.reshape(-1, 1)

# === Step 2: 拟合 HMM 模型 ===
model = GaussianHMM(n_components=3, covariance_type="full", random_state=42)
model.fit(sp500_returns)

# === Step 3: 获取每天的状态概率（低/中/高波动）
state_probs = model.predict_proba(sp500_returns)
state_preds = model.predict(sp500_returns)

# === Step 4: 输出为 DataFrame
dates = sp500.index[1:]  # drop first nan
hmm_df = pd.DataFrame(state_probs, columns=["Low_Vol", "Med_Vol", "High_Vol"], index=dates)
hmm_df["Regime_Label"] = state_preds

# === Step 5: 提取频率样本
hmm_daily = hmm_df.copy()
hmm_weekly = hmm_df[hmm_df.index.weekday == 0]
hmm_monthly = hmm_df[hmm_df.index.day == 1]

# === Step 6: 保存 & 显示示例
hmm_daily.to_csv("hmm_states_daily.csv")
hmm_weekly.to_csv("hmm_states_weekly.csv")
hmm_monthly.to_csv("hmm_states_monthly.csv")

print("HMM 市场状态（每日前5行）:")
print(hmm_daily.head())

# 可视化（可选）
# hmm_daily[["Low_Vol", "Med_Vol", "High_Vol"]].plot(figsize=(12,4), title="HMM Volatility Regime Probabilities")


[*********************100%***********************]  1 of 1 completed


HMM 市场状态（每日前5行）:
             Low_Vol   Med_Vol      High_Vol  Regime_Label
Date                                                      
2020-01-03  0.988551  0.011449  6.812180e-15             0
2020-01-06  0.011849  0.988142  8.988288e-06             1
2020-01-07  0.984131  0.015557  3.116907e-04             0
2020-01-08  0.015986  0.983896  1.173631e-04             1
2020-01-09  0.980068  0.019585  3.468418e-04             0


In [18]:
import pandas as pd
import numpy as np
import cvxpy as cp

# === DCP 失败计数器 ===
dcp_failed_count = {"D": 0, "W": 0, "M": 0}

# === 多因子 Markowitz 优化器 ===
def markowitz_multifactor(mu, cov, w_prev, sentiment_vec, hmm_vec, cvar_scalar,
                          gamma_hold=0.01, gamma_trade=0.01,
                          lambda_var=1.0, lambda_sent=10.0, lambda_hmm=5.0,
                          verbose=False, freq="D"):
    global dcp_failed_count

    n_assets = len(mu)
    w = cp.Variable(n_assets)
    z = w - w_prev

    expected_return = mu @ w
    holding_cost = cp.norm1(cp.pos(-w))
    trading_cost = cp.norm1(z) + cp.sum_squares(z)
    gamma_trade_eff = gamma_trade * (1 + hmm_vec[2])

    sentiment_penalty = cp.sum(cp.multiply(sentiment_vec, cp.square(w)))
    hmm_penalty = lambda_hmm * cp.sum_squares(w)
    variance_penalty = cp.quad_form(w, cp.psd_wrap(cov))

    objective = cp.Maximize(
        expected_return
        - gamma_hold * holding_cost
        - gamma_trade_eff * trading_cost
        - lambda_var * variance_penalty
        - lambda_sent * sentiment_penalty
        - hmm_penalty
    )

    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        w <= 1
    ]

    problem = cp.Problem(objective, constraints)

    if not problem.is_dcp():
        dcp_failed_count[freq] += 1
        if verbose:
            print("DCP check failed.")
        return None

    try:
        problem.solve()
        if w.value is None:
            if verbose:
                print("Solver failed. Returning previous weights.")
            return None
        return w.value
    except Exception as e:
        if verbose:
            print(f"Optimization error: {e}")
        return None

# === 回测模拟函数 ===
def simulate_portfolio(returns, sentiment, hmm, cvar, rebalance_freq='M', lookback=60):
    returns = returns.dropna(axis=1)
    sentiment = sentiment[returns.columns]
    dates = returns.index
    n_assets = returns.shape[1]
    w_prev = np.ones(n_assets) / n_assets
    port_returns = []

    for i in range(lookback, len(returns)):
        date = dates[i]
        if rebalance_freq == 'M' and date.day != 1:
            port_returns.append(returns.iloc[i].values @ w_prev)
            continue
        if rebalance_freq == 'W' and date.weekday() != 0:
            port_returns.append(returns.iloc[i].values @ w_prev)
            continue

        window = returns.iloc[i - lookback:i]
        mu = window.mean().values * 252
        cov = window.cov().values * 252
        try:
            sentiment_vec = sentiment.loc[date].values
            hmm_vec = hmm.loc[date][["Low_Vol", "Med_Vol", "High_Vol"]].values
            cvar_scalar = cvar.loc[date]
        except:
            port_returns.append(returns.iloc[i].values @ w_prev)
            continue

        try:
            w_opt = markowitz_multifactor(mu, cov, w_prev, sentiment_vec, hmm_vec,
                                          cvar_scalar, verbose=True, freq=rebalance_freq)
            if w_opt is not None:
                w_prev = w_opt
        except Exception as e:
            print(f"Optimization failed on {date.date()}: {e}")

        port_returns.append(returns.iloc[i].values @ w_prev)

    return pd.Series(port_returns, index=returns.index[lookback:])

# === 年化收益、波动率、Sharpe Ratio ===
def performance_metrics(r, risk_free_rate=0.02):
    ann_return = (1 + r.mean())**252 - 1
    ann_vol = r.std() * np.sqrt(252)
    sharpe_ratio = (ann_return) / ann_vol
    return ann_return, ann_vol, sharpe_ratio

# === 加载数据 ===
returns = pd.read_csv("sector_portfolio_returns.csv", index_col=0, parse_dates=True)
sentiment_daily = pd.read_csv("sector_sentiment_daily_scaled.csv", index_col=0, parse_dates=True)
sentiment_weekly = pd.read_csv("sector_sentiment_weekly_scaled.csv", index_col=0, parse_dates=True)
sentiment_monthly = pd.read_csv("sector_sentiment_monthly_scaled.csv", index_col=0, parse_dates=True)
hmm_daily = pd.read_csv("hmm_states_daily.csv", index_col=0, parse_dates=True)
hmm_weekly = pd.read_csv("hmm_states_weekly.csv", index_col=0, parse_dates=True)
hmm_monthly = pd.read_csv("hmm_states_monthly.csv", index_col=0, parse_dates=True)
cvar = pd.read_csv("cvar_frequencies.csv", index_col=0, parse_dates=True)

# === 回测（D/W/M）
for freq, sentiment, hmm in zip(['D', 'W', 'M'],
                                [sentiment_daily, sentiment_weekly, sentiment_monthly],
                                [hmm_daily, hmm_weekly, hmm_monthly]):
    print(f"\n====== {freq}-rebalancing 回测开始 ======")
    cvar_col = f"CVaR_{'Daily' if freq == 'D' else 'Weekly' if freq == 'W' else 'Monthly'}"
    r = simulate_portfolio(
        returns,
        sentiment,
        hmm,
        cvar[cvar_col],
        rebalance_freq=freq,
        lookback=60
    )
    ann_return, ann_vol, sharpe = performance_metrics(r)
    print(f"多因子 MVO 表现 ({freq}-rebalancing):")
    print(f"Annual Return: {ann_return:.4f}")
    print(f"Annual Volatility: {ann_vol:.4f}")
    print(f"Sharpe Ratio: {sharpe:.4f}")
    print(f"Total DCP check failed: {dcp_failed_count[freq]}")



====== D-rebalancing 回测开始 ======
多因子 MVO 表现 (D-rebalancing):
Annual Return: 0.2658
Annual Volatility: 0.1681
Sharpe Ratio: 1.5816
Total DCP check failed: 0

====== W-rebalancing 回测开始 ======
多因子 MVO 表现 (W-rebalancing):
Annual Return: 0.2680
Annual Volatility: 0.1749
Sharpe Ratio: 1.5327
Total DCP check failed: 0

====== M-rebalancing 回测开始 ======
多因子 MVO 表现 (M-rebalancing):
Annual Return: 0.3127
Annual Volatility: 0.1749
Sharpe Ratio: 1.7878
Total DCP check failed: 0
